In [2]:
# Setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import sys
sys.path.append('/Users/danielruiz/Dropbox/Grad School/Fall 2018/CS273A/Project/')
import mltools as ml
import data_loader
import warnings
warnings.filterwarnings('ignore')

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

In [3]:
# load data (provided method)
train_data, valid_data = data_loader.load_train_data('Data/adult.data.txt', valid_rate=0.1, is_df=True)
test_data = data_loader.load_test_data('Data/adult.test.txt', is_df=True)

In [4]:
# specify columns labels
features = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation',
                'relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country',
                'income']
train_data.columns = features
valid_data.columns = features
test_data.columns = features

# drop unnecessary features (fnlwgt, education)
train_data = train_data.drop(columns = ['fnlwgt', 'education'])
valid_data = valid_data.drop(columns = ['fnlwgt', 'education'])
test_data = test_data.drop(columns = ['fnlwgt', 'education'])

In [11]:
# numerical processing pipeline
num_features = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
num_transformer = Pipeline(steps = [('scaler', StandardScaler())]) # currently just the scaler

In [46]:
# categorical processing pipeline
cat_features = ['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 
                'native-country']
cat_transformer = Pipeline(steps = [('imputer', SimpleImputer(missing_values = ' ?',strategy = 'most_frequent')),
                                    ('onehot', OneHotEncoder(handle_unknown = 'ignore'))])

In [47]:
# create preprocessor containing above pipelines
preprocessor = ColumnTransformer(transformers=[('num', num_transformer, num_features), 
                                               ('cat', cat_transformer, cat_features)])

In [48]:
# create a new pipeline containign the preprocessor and a classifier
full_pipe = Pipeline(steps=[('preprocessor', preprocessor),('classifier', LogisticRegression(solver='lbfgs'))])

In [49]:
# convert target values to binary categories
train_data = train_data.replace(to_replace = ' >50K', value = 1)
train_data = train_data.replace(to_replace = ' <=50K', value = 0)
valid_data = valid_data.replace(to_replace = ' >50K', value = 1)
valid_data = valid_data.replace(to_replace = ' <=50K', value = 0)
test_data = test_data.replace(to_replace = ' >50K', value = 1)
test_data = test_data.replace(to_replace = ' <=50K', value = 0)

In [50]:
# remove target values
X = train_data.drop(columns = 'income')
Y = train_data['income']

In [51]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .2)

In [52]:
full_pipe.fit(X_train,Y_train)

Pipeline(memory=None,
     steps=[('preprocessor', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('num', Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True))]), ['age', 'education-num', 'capital...enalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False))])

In [54]:
print('Model Score: ' + str(full_pipe.score(X_test, Y_test)))

Model Score: 0.8548524646085621


In [89]:
# count nan values in each column
null_columns = train_data_nan.columns[train_data_nan.isnull().any()]
train_data_nan[null_columns].isnull().sum()

workclass         1662
occupation        1669
native-country     498
dtype: int64

In [90]:
# remove rows containing nan values
tr_clean = train_data_nan.dropna()

In [91]:
tr_clean.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,27175.000000,2.717500e+04,27175.000000,27175.000000,27175.000000,27175.000000
mean,38.430874,1.894224e+05,10.111279,1111.609457,89.200626,40.931923
std,13.117079,1.058884e+05,2.554387,7503.118210,405.936240,11.942297
min,17.000000,1.376900e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.173630e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.779550e+05,10.000000,0.000000,0.000000,40.000000
75%,47.000000,2.369935e+05,13.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000
